In [1]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fad52dadf90>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fad5318b820>


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2993, 0.2859, 0.3126],
        [0.2993, 0.2859, 0.3126],
        [0.2993, 0.2859, 0.3126],
        [0.2993, 0.2859, 0.3126],
        [0.2993, 0.2859, 0.3126]])
tensor([0.2993, 0.2859, 0.3126])


In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [8]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300808  [    0/60000]
loss: 2.293719  [ 6400/60000]
loss: 2.271294  [12800/60000]
loss: 2.274005  [19200/60000]
loss: 2.265560  [25600/60000]
loss: 2.223298  [32000/60000]
loss: 2.243927  [38400/60000]
loss: 2.208505  [44800/60000]
loss: 2.205343  [51200/60000]
loss: 2.179200  [57600/60000]
Test Error: 
 Accuracy: 34.8%, Avg loss: 2.172230 

Epoch 2
-------------------------------
loss: 2.175321  [    0/60000]
loss: 2.174171  [ 6400/60000]
loss: 2.113389  [12800/60000]
loss: 2.140952  [19200/60000]
loss: 2.112135  [25600/60000]
loss: 2.027094  [32000/60000]
loss: 2.076331  [38400/60000]
loss: 1.995318  [44800/60000]
loss: 1.997340  [51200/60000]
loss: 1.945535  [57600/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.935125 

Epoch 3
-------------------------------
loss: 1.949883  [    0/60000]
loss: 1.938031  [ 6400/60000]
loss: 1.820712  [12800/60000]
loss: 1.877434  [19200/60000]
loss: 1.794262  [25600/60000]
loss: 1.707519  [32000/600